In [1]:
from collections import Counter,defaultdict
import csv
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from pprint import pprint
import json
from bs4 import BeautifulSoup
import sys
import os
import ast
from gensim.models.doc2vec import Doc2Vec,LabeledSentence

#os.environ['KERAS_BACKEND']='theano'
import keras
import keras.backend as K
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Merge, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.callbacks import Callback
import keras.objectives


from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score, classification_report,jaccard_similarity_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import learning_curve
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
from sklearn.naive_bayes import MultinomialNB

import re

C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
texts = []
labels = []
urls = []

with open('parlementData_2001-2017.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='&',quotechar='|')
    for row in spamreader:
        urls.append(row[0])
        texts.append(row[1])
        labels.append(ast.literal_eval(row[2]))
        
munTexts = []
munLabels = []
munNumber = []

with open('municipality.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='&',quotechar='|')
    for row in spamreader:
        munNumber.append(row[0])
        munTexts.append(row[1])
        munLabels.append(ast.literal_eval(row[2]))
        
with open('municipalityFam.csv', 'r') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='&',quotechar='|')
    for row in spamreader:
        munNumber.append(row[0])
        munTexts.append(row[1])
        munLabels.append(ast.literal_eval(row[2]))

In [5]:
TEST_SET = 0.15
nb_test_samples = int(TEST_SET * len(labels))

MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 160

lb = MultiLabelBinarizer()
labelsCNN = lb.fit_transform(labels)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labelsCNN.shape)

x_train = data[:-nb_test_samples]
y_train = labelsCNN[:-nb_test_samples]
x_test = data[-nb_test_samples:]
y_test = labelsCNN[-nb_test_samples:]

xTrain = texts[:-nb_test_samples]
xTest = texts[-nb_test_samples:]

transformer = TfidfVectorizer(smooth_idf=False, min_df=0.00001, max_df=0.2, sublinear_tf=True)
    
xTrain = transformer.fit_transform(xTrain)
xTest = transformer.transform(xTest)

print('Class distribution in training and validation set ')
print(y_train.sum(axis=0))
print(y_test.sum(axis=0))

xMun = transformer.transform(munTexts)
yMun = lb.transform(munLabels)

Found 392519 unique tokens.
Shape of data tensor: (52397, 1000)
Shape of label tensor: (52397, 17)
Class distribution in training and validation set 
[2916 3559 6695 3454 1079 6047 1815 3192 4703 2935 8692 8222 1637 1591
 3651 2902 9145]
[ 523  632 1206  591  181 1070  330  513  873  574 1557 1437  257  294
  698  519 1595]


In [59]:
def evaluation(yVal,yPred):
    print("F1 is "        + str(round(f1_score(yVal, yPred, average="micro"),3)) + 
        ", Precision is " + str(round(precision_score(yVal, yPred, average="micro"),3)) +
        ", Recall is "    + str(round(recall_score(yVal, yPred, average="micro"),3)) +
        ", Accuracy is "  + str(round(accuracy_score(yVal, yPred),3)))


def evaluationWithBestThreshold(clf, xMun, yMun, Thresholds = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]):
    bestF1 = 0
    bestThreshold = 0
    yPred = clf.predict_proba(xMun)
    for threshold in [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]:
        F1 = f1_score(yMun, (yPred > threshold).astype(int), average="micro")
        if F1 > bestF1: 
            bestF1 = F1
            bestThreshold = threshold
    evaluation(yMun,(yPred > bestThreshold).astype(int))


### BASELINES

In [60]:
clf = MultinomialNB(fit_prior= False)
clf = OneVsRestClassifier(clf).fit(xTrain, y_train)

print("Naive Bayes - Parliament")
evaluationWithBestThreshold(clf, xTest, y_test)

print("\n Naive Bayes - Municipality")
evaluationWithBestThreshold(clf, xMun, yMun)

Naive Bayes - Parliament
F1 is 0.283, Precision is 0.892, Recall is 0.168, Accuracy is 0.158

 Naive Bayes - Municipality
F1 is 0.037, Precision is 0.085, Recall is 0.024, Accuracy is 0.307


C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
clf = SGDClassifier(alpha = 1e-05, loss = 'hinge', penalty = 'l2', random_state=42, max_iter=10, tol=None)
clf = OneVsRestClassifier(clf).fit(xTrain, y_train)

print("SVM - Parliament")
yPred = clf.predict(xTest)
evaluation(y_test,yPred)

print("\n SVM - Municipality")
yPred = clf.predict(xMun)
evaluation(yMun,yPred)

In [57]:
clf = LogisticRegression(C= 10, penalty= 'l2', solver= 'liblinear',dual=False,max_iter=250)
clf = OneVsRestClassifier(clf).fit(xTrain, y_train)

print("Logistic Regression - Parliament")
evaluationWithBestThreshold(clf, xTest, y_test)

print("\n Logistic Regression - Municipality")
evaluationWithBestThreshold(clf, xMun, yMun)

Logistic Regression - Parliament
F1 is 0.78, Precision is 0.781, Recall is 0.7789, Accuracy is 0.5025

 Logistic Regression - Municipality
F1 is 0.2781, Precision is 0.2226, Recall is 0.3706, Accuracy is 0.1376


In [ ]:
clf = RandomForestClassifier(criterion= 'entropy', max_depth= 100, n_estimators= 30)
clf = OneVsRestClassifier(clf).fit(xTrain, y_train)

print("Random Forest - Parliament")
evaluationWithBestThreshold(clf, xTest, y_test)

print("\n Random Forest - Municipality")
evaluationWithBestThreshold(clf, xTest, y_test)

### KIM'S CNN + SPLITUPS

Parameters:
W2V:                Self-trained, imported
Embedding:          Static, Dynamic
split:              true, false
multi-filter sizes: true, false
filter sizes:       3,5,7,11
filter sizes:       [2,3,4], [3,4,5], [7,8,9]

2 * 2 * 2 * 2 * 4 + 2 * 2 * 2 * 2 * 3 = 112 versions

In [7]:
GLOVE_DIRS   = [r"C:\Users\PC-Axel\Documents\Codeer projecten\Word2Vec Vectoren\Nederlandse word2vec\combined-160.txt",
                r"C:\Users\PC-Axel\Documents\GitHub\thesis\Code\Data preparation\zelfgemaakte-w2v.txt"]
TRAINABLE    = [True, False]
SPLIT        = [True, False]
MULTIFILTER  = [True, False]
FILTERS      = [3,5,7,11]
MULTIFILTERS = [[2,3,4], [3,4,5], [7,8,9]]

In [8]:
def fmeasure(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))

    p = true_positives / (predicted_positives + K.epsilon())
    r = true_positives / (possible_positives + K.epsilon())

    beta = 1 # fmeasure
    bb = beta**2
    fbeta_score = (1 + bb) * (p * r) / (bb * p + r + K.epsilon())
    return fbeta_score

In [9]:
def getEmbedding(GLOVE_DIR):
    embeddings_index = {}
    f = open(GLOVE_DIR, encoding="utf8")
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()
    return embeddings_index

In [15]:
def testVersionCNN(embeddings_index, RETRAIN, FILTER, FILTERSIZE,filepath):
    embedding_matrix = np.random.random((len(word_index) + 1, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(len(word_index) + 1,
                                EMBEDDING_DIM,
                                weights=[embedding_matrix],
                                input_length=MAX_SEQUENCE_LENGTH,
                                trainable=RETRAIN)
    
    sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    
    if FILTER:
        convs = []
        for fsz in FILTERSIZE:
            l_conv = Conv1D(filters=128,kernel_size=fsz,activation='relu')(embedded_sequences)
            l_pool = MaxPooling1D(5)(l_conv)
            convs.append(l_pool)
        l_previous = Merge(mode='concat', concat_axis=1)(convs)
    else:
        l_cov1 = Conv1D(128, FILTERSIZE, activation='relu')(embedded_sequences)
        l_previous = MaxPooling1D(5)(l_cov1)
    
    l_cov2 = Conv1D(128, 5, activation='relu')(l_previous)
    l_pool2 = MaxPooling1D(5)(l_cov2)
    l_cov3 = Conv1D(128, 5, activation='relu')(l_pool2)
    if FILTER:
        l_pool3 = MaxPooling1D(int(l_cov3.shape[1]))(l_cov3)  # global max pooling
    else:
        l_pool3 = MaxPooling1D(int(l_cov3.shape[1]))(l_cov3)
    l_flat = Flatten()(l_pool3)
    l_drop = Dropout(0.5)(l_flat)
    l_dense = Dense(128, activation='relu')(l_drop)
    preds = Dense(labelsCNN.shape[1], activation='sigmoid')(l_dense)

    modelCNN = Model(sequence_input, preds)
    modelCNN.compile(loss='binary_crossentropy',
                  optimizer='rmsprop',
                  metrics=[keras.metrics.categorical_accuracy, fmeasure])
    filepath= "best_models\CNN_" + filepath
    checkpoint = ModelCheckpoint(filepath, monitor="val_fmeasure", verbose=0, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    return modelCNN, callbacks_list

In [22]:
# Without split
for GLOVE_DIR,a in zip(GLOVE_DIRS,["W2V_loaded_","W2V_own_"]):
    embeddings_index = getEmbedding(GLOVE_DIR)
    for RETRAIN,b in zip(TRAINABLE,["dynamic_", "static_"]):
        for FILTER,c in zip(MULTIFILTER,["multi_","single_"]):
            if FILTER:
                for FILTERSIZE,d in zip(MULTIFILTERS,["234","345","789"]):
                    modelCNN, callbacks_list = testVersionCNN(embeddings_index, RETRAIN, FILTER, FILTERSIZE, a+b+c+d)
                    modelCNN.fit(x_train, y_train, validation_data=(x_test, y_test),
                                 epochs=10, batch_size=128, callbacks=callbacks_list)
            else:
                for FILTERSIZE,d in zip(FILTERS,["3","5","7","11"]):
                    modelCNN, callbacks_list = testVersionCNN(embeddings_index, RETRAIN, FILTER, FILTERSIZE, a+b+c+d)
                    modelCNN.fit(x_train, y_train, validation_data=(x_test, y_test),
                                 epochs=10, batch_size=128, callbacks=callbacks_list)
# With split


C:\Users\PC-Axel\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:24: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


Train on 44538 samples, validate on 7859 samples
Epoch 1/10
44538/44538 [==============================] - 1144s 26ms/step - loss: 0.2844 - categorical_accuracy: 0.2036 - fmeasure: 0.1322 - val_loss: 0.2285 - val_categorical_accuracy: 0.3675 - val_fmeasure: 0.4155

Epoch 00001: val_fmeasure improved from -inf to 0.41549, saving model to best_models\CNN_W2V_loaded_dynamic_multi_234
Epoch 2/10
32128/44538 [====================>.........] - ETA: 4:59 - loss: 0.2072 - categorical_accuracy: 0.4263 - fmeasure: 0.4392

KeyboardInterrupt: 

### PAR2VEC

### KALCHBRENNER CNN